In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QCheckBox, QSpinBox
import serial
import time
import base64
#import keyboard

# UI 

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QCheckBox, QSpinBox

class MotorControlerUI(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.ui_events = []

    def initUI(self):
        # Create buttons s1 to s6
        self.buttons = []
        button_pos =[(50,50),(50,100),(50,150),(50,200),(50,250),(50,300)]
        for i in range(1, 7):
            button = QPushButton(f'S{i}', self)
            button.clicked.connect(self.buttonClicked)
            button.move(button_pos[i-1][0],button_pos[i-1][1])
            self.buttons.append(button)

        #labels for s1->s6
        self.labels = []
        label_pos = [(bp[0]*4,bp[1]) for bp in button_pos]
        for i in range(1,7):
            label = QLabel(f"Label{i}",self)
            label.move(label_pos[i-1][0],label_pos[i-1][1])
            label.setText("0")
            label.setFixedWidth(50)
            self.labels.append(label)

        #spinBox for s1->s6
        self.spinboxs = []
        spinbox_pos = [(bp[0]*6,bp[1]) for bp in button_pos]
        for i in range(1,7):
            spinbox = QSpinBox(self)
            spinbox.setObjectName(f"s{i}")
            spinbox.setRange(-500,500)
            spinbox.setFixedWidth(150)
            spinbox.move(spinbox_pos[i-1][0],spinbox_pos[i-1][1])
            self.spinboxs.append(spinbox)

        self.labelInfo = QLabel("- = ClockWise",self)
        self.labelInfo.move(spinbox_pos[0][0],spinbox_pos[0][1]-25)

        self.labelPos = QLabel("MotorPosition",self)
        self.labelPos.move(label_pos[0][0],label_pos[0][1]-25)

        self.setWindowTitle('Motor controler')
        self.setGeometry(300, 300, 500, 500)  # Set window position and size


    #UI EVENTS
    def buttonClicked(self): #button click (adds spinbox to label)
        sender = self.sender()
        index = int(sender.text()[1]) - 1
        spinbox = self.spinboxs[index]
        change_val = spinbox.value()
        label = self.labels[index]
        cur_val = int(label.text())
        new_val = change_val+cur_val
        label.setText(str(new_val))
        self.ui_events.append([index,change_val,f'Button {sender.text()} clicked. With Value {change_val}: {cur_val} -> {new_val}'])

#Handle events
def handleUIEvents(ui_widget,mc):
    while ui_widget.ui_events:
        event = ui_widget.ui_events.pop(0)
        print(event)

        mc[event[0]] = event[1]
    app.processEvents()

C:\Users\DGaard\AppData\Local\Temp\ipykernel_26840\2525193655.py:4: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class MotorControlerUI(QWidget):


# Serial protocol

In [3]:
#initial protocol from https://forum.arduino.cc/t/serial-input-basics-updated/382007/2
# and https://forum.arduino.cc/t/pc-arduino-comms-using-python-updated/574496 

startMarker = '<'
endMarker = '>'
dataStarted = False
dataBuf = ""
messageComplete = False

#========================
#========================
    # the functions

def setupSerial(baudRate, serialPortName):
    global  serialPort
    serialPort = serial.Serial(port= serialPortName, baudrate = baudRate, timeout=0, rtscts=True)
    print("Serial port " + serialPortName + " opened  Baudrate " + str(baudRate))
    waitForArduino()

#========================

def sendToArduino(stringToSend):
    # this adds the start- and end-markers before sending
    global startMarker, endMarker, serialPort
    
    stringWithMarkers = (startMarker)
    stringWithMarkers += stringToSend
    stringWithMarkers += (endMarker)

    serialPort.write(stringWithMarkers.encode('utf-8')) # encode needed for Python3


#==================

def recvLikeArduino():

    global startMarker, endMarker, serialPort, dataStarted, dataBuf, messageComplete

    if serialPort.inWaiting() > 0 and messageComplete == False:
        x = serialPort.read().decode("utf-8") # decode needed for Python3
        
        if dataStarted == True:
            if x != endMarker:
                dataBuf = dataBuf + x
            else:
                dataStarted = False
                messageComplete = True
        elif x == startMarker:
            dataBuf = ''
            dataStarted = True
    
    if (messageComplete == True):
        messageComplete = False
        return dataBuf
    else:
        return "XXX" 

#==================

def waitForArduino():
    # wait until the Arduino sends 'Arduino is ready' - allows time for Arduino reset
    # it also ensures that any bytes left over from a previous message are discarded
    print("Waiting for Arduino to reset")
     
    msg = ""
    while msg.find("Arduino is ready") == -1:
        msg = recvLikeArduino()
        if not (msg == 'XXX'): 
            print(msg)

# Messaging protocol

In [4]:
#<msg> (where "<" starts a msg and ">" ends 1)

#start <
#first char is used for identifying the type of msg. M=reltive move, R=reset
#next 4x6 chars are used for the amount you want the steppers to move 2signed bytes pr stepper
#4bytes for the amount of time to complete the moves
#end >

def num_to_hex(num):
  if num > 32767  or num < -32768:
    raise TypeError(f"Number {num} is does not fit withing 4 char hex")
  
  # Convert negative numbers to two's complement representation
  if num < 0:
    num = (1 << 16) + num
  
  hex_value = hex(num & 0xFFFF)[2:].zfill(4)  # Convert to hexadecimal and pad with zeros
  return hex_value

def hex_to_num(hex_string):
  if not isinstance(hex_string, str) or len(hex_string) != 4:
    TypeError(f"Input must be a string of 4 characters")
  
  num = int(hex_string, 16)
  # Handle negative numbers
  if num >= 0x8000:
      num -= 1 << 16
  return num

#convert array of relative motor steps to a string that can be sent to the arduino
#current types: R and M
def encode_motor_msg(motor_steps,timer,msg_type):
  if not(msg_type in ["M","R"]):
    TypeError(f"Unknown msg_type: {msg_type}")
  
  if len(motor_steps) != 6:
    TypeError(f"Expected 6 motor values got: {motor_steps}")
  
  motor_hex = ""
  for ms in motor_steps:
    motor_hex += num_to_hex(ms)
  
  time_hex = num_to_hex(timer)

  return msg_type + motor_hex + time_hex

#decode motor message

def decode_motor_msg(motor_msg):
  msg_type = motor_msg[0]
  motor_hex = motor_msg[1:25]
  timer_hex = motor_msg[25:29]

  motor_vals = []
  for i in range(0,24,4):
    motor_vals.append(hex_to_num(motor_hex[i:i+4])) 
  

  return msg_type, motor_vals, hex_to_num(timer_hex)


# Examples:
numbers = [-1234, 32767, 9012, 3456, -7890, 1234]

for i in numbers:
  hexval = num_to_hex(i)
  print(hexval,hex_to_num(hexval))

fullEncode = encode_motor_msg(numbers,1000,"M")
print(fullEncode,decode_motor_msg(fullEncode))

fb2e -1234
7fff 32767
2334 9012
0d80 3456
e12e -7890
04d2 1234
Mfb2e7fff23340d80e12e04d203e8 ('M', [-1234, 32767, 9012, 3456, -7890, 1234], 1000)


# Main loop


In [5]:
setupSerial(115200, "COM3")
count = 0
prevTime = time.time()

motor_vals = [0,0,0,0,0,0]
motor_change = [0,0,0,0,0,0]

if __name__ == '__main__':
    app = QApplication(sys.argv)
    mcUI = MotorControlerUI()
    mcUI.show()

    # Your main loop
    while True:
        # Receive and handle messages from the microcontroller 
        arduinoReply = recvLikeArduino() # check for a reply
        if not (arduinoReply == 'XXX'):
            print ("Time %s  Reply %s" %(time.time(), arduinoReply))
    
        # Handle UI events
        handleUIEvents(mcUI,motor_change)

        # Respond to the microcontroller
        if motor_change != [0,0,0,0,0,0]:
            motor_vals = [a+b for a,b in zip(motor_vals,motor_change)] #update total motor values
            enc_msg = encode_motor_msg(motor_change,1000,"M")
            sendToArduino(enc_msg)
            print("motorchange:",motor_change,"msg:", enc_msg)

            motor_change = [0,0,0,0,0,0]


        #elif time.time() - prevTime > 1.0: # send a message at intervals
        #    sendToArduino("this is a test " + str(count))
        #    prevTime = time.time()
        #    count += 1
    
    sys.exit(app.exec_()) #independently run the program


Serial port COM3 opened  Baudrate 115200
Waiting for Arduino to reset
Arduino is ready
[0, 9, 'Button S1 clicked. With Value 9: 0 -> 9']
motorchange: [9, 0, 0, 0, 0, 0] msg: M00090000000000000000000003e8
Time 1712933290.4948826  Reply This just in ... M00090000000000000000000003e8   M   9   0   0   0   0   0   1000   7649
[1, -6, 'Button S2 clicked. With Value -6: 0 -> -6']
motorchange: [0, -6, 0, 0, 0, 0] msg: M0000fffa000000000000000003e8
Time 1712933301.6388254  Reply This just in ... M0000fffa000000000000000003e8   M   0   -6   0   0   0   0   1000   18792
[2, 0, 'Button S3 clicked. With Value 0: 0 -> 0']
[2, 3, 'Button S3 clicked. With Value 3: 0 -> 3']
motorchange: [0, 0, 3, 0, 0, 0] msg: M00000000000300000000000003e8
Time 1712933365.1200867  Reply This just in ... M00000000000300000000000003e8   M   0   0   3   0   0   0   1000   82262
[2, 3, 'Button S3 clicked. With Value 3: 3 -> 6']
motorchange: [0, 0, 3, 0, 0, 0] msg: M00000000000300000000000003e8
Time 1712933369.3180535  Rep